In [2]:
from selenium import webdriver
import time
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import openpyxl as op
import random
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
#↑發生錯誤時使用的module

driver = webdriver.Chrome("C:\\Users\\User\\chromedriver.exe")
driver.get("https://www.booking.com/searchresults.zh-tw.html?label=booking-name-yefrPbbyS*FIINHgyCnmNgS267725091255%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9040379%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YfqnDqqG8nt1O4nYvDr1lms&sid=793d0395067d9ba3d43b2cd49ea048ce&aid=376396&src=searchresults&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.zh-tw.html%3Faid%3D376396%3Blabel%3Dbooking-name-yefrPbbyS%252AFIINHgyCnmNgS267725091255%253Apl%253Ata%253Ap1%253Ap22%252C563%252C000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-65526620%253Alp9040379%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9YfqnDqqG8nt1O4nYvDr1lms%3Bsid%3D793d0395067d9ba3d43b2cd49ea048ce%3Btmpl%3Dsearchresults%3Bcheckin%3D2022-05-07%3Bcheckout%3D2022-05-08%3Bclass_interval%3D1%3Bdest_id%3D-2637882%3Bdest_type%3Dcity%3Bdtdisc%3D0%3Bgroup_adults%3D2%3Bgroup_children%3D0%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Bno_rooms%3D1%3Boffset%3D0%3Bpostcard%3D0%3Braw_dest_type%3Dcity%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bsb_travel_purpose%3Dleisure%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrc%3Dsearchresults%3Bsrc_elem%3Dsb%3Bsrpvid%3D5fe4260dd8400063%3Bss%3D%25E5%258F%25B0%25E5%258C%2597%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Bssne%3D%25E5%258F%25B0%25E5%258C%2597%3Bssne_untouched%3D%25E5%258F%25B0%25E5%258C%2597%26%3B&ss=%E5%8F%B0%E5%8C%97&is_ski_area=0&ssne=%E5%8F%B0%E5%8C%97&ssne_untouched=%E5%8F%B0%E5%8C%97&city=-2637882&checkin_year=2022&checkin_month=5&checkin_monthday=07&checkout_year=2022&checkout_month=5&checkout_monthday=08&group_adults=2&group_children=0&no_rooms=1&sb_travel_purpose=business&from_sf=1&nflt=ht_id%3D204")

"""
# 獲得當前所有開啟的視窗的控制代碼
all_handles = driver.window_handles
for handle in all_handles:
 driver.switch_to.window(handle)
 driver.close()
 time.sleep(2)
"""
# 網頁
desired_capabilities = DesiredCapabilities.CHROME
desired_capabilities["pageLoadStrategy"] = "eager" 

ex1 = op.Workbook()
ex1.active.title="2022-05-07"   #修改工作表名稱:2022-月月-日日
sheet1=ex1.active  #工作變數:sheet1
sheet1.append(["資料來源","飯店名稱","地址","入住日期","房型","價格(含早餐)",
               "住宿地點","清潔程度","舒適程度","設施與設備","員工素質與服務"])
#↑欄位名稱

for i in range(19):  #平均每一天有14~18頁的資訊，可以手動修改
    # 1page的資訊
    for hotel in driver.find_elements_by_css_selector('div[data-testid="title"]'):
        #hotel_info為list，把每一間飯店的資訊append進這個list
        hotel_info = ["Booking"]   #第一個內容是資料來源，所以先給它第一個值是"Booking"
        hotel.click()   #點擊該飯店，目的:進入該間飯店的頁面
        time.sleep(10)  #等待頁面跑的時間
        window_num = driver.window_handles   #window分頁
        driver.switch_to.window(window_num[1])  #切換分頁到新視窗
        time.sleep(5)  #給他一點時間跑網頁內容
        # 飯店名稱
        name = driver.find_element_by_id("hp_hotel_name_reviews").get_attribute("innerText")
        hotel_info.append(name)  #增加到hotel_info這個list
        # 地址
        address = driver.find_element_by_css_selector('.hp_address_subtitle')
        address = address.text
        #先初步整理地址的內容
        if address[2]!="市":     # 判斷鄉鎮市區
            address = address[:2]+"市"+address[2:]
        if address[5]!="區":     # 判斷鄉鎮市區
            address = address[:5]+"區"+address[5:]
        hotel_info.append(address)
        time.sleep(2)  #給2秒的時間網頁讀取
        hotel_info.append("2022-05-07")
        # 房型
        #曾經出現過錯誤，網頁內沒有房型這個資料，所以給try&except的函式除錯
        try:
            roomtype = driver.find_element_by_css_selector('.hprt-roomtype-icon-link') 
            hotel_info.append(roomtype.text)
        except:
            driver.close()
            driver.switch_to.window(window_num[0])
            continue
        # 價格:使用try&except的原因跟房型一樣
        try:
            price = driver.find_elements_by_css_selector('span[class="prco-valign-middle-helper"]')[1]
            price = price.text.replace("TWD ","")
            hotel_info.append(price.replace(",",""))
        except:
            driver.close()
            driver.switch_to.window(window_num[0])
            continue
        time.sleep(2)
        # 各項評分標準: 住宿地點,清潔程度,舒適程度,設施與設備,員工素質與服務
        try:
            staff = driver.find_elements_by_css_selector("span.c-score-bar__score")[0].get_attribute("innerText")
            location = driver.find_elements_by_css_selector("span.c-score-bar__score")[1].get_attribute("innerText")
            cleanliness = driver.find_elements_by_css_selector("span.c-score-bar__score")[2].get_attribute("innerText")
            comfort = driver.find_elements_by_css_selector("span.c-score-bar__score")[3].get_attribute("innerText")
            facility = driver.find_elements_by_css_selector("span.c-score-bar__score")[4].get_attribute("innerText")
            time.sleep(2)
            hotel_info.append(location)  #住宿地點
            hotel_info.append(cleanliness)  #清潔程度
            hotel_info.append(comfort)  #舒適程度
            hotel_info.append(facility)  #設備與設施
            hotel_info.append(staff)  #員工素質與服務
            #為了統一順序方便之後做資料統整&清理，所以在這裡調整資料append的順序
            sheet1.append(hotel_info)   #把單筆飯店資料增加到excel裡
        except:
            driver.close()
            driver.switch_to.window(window_num[0])
            continue
        ex1.save("20220427_Booking-0507.xlsx")  #儲存excel
        # 關閉分頁
        driver.close()
        driver.switch_to.window(window_num[0])  #切回原本的分頁
        time.sleep(2)

    # 下一頁
    next_page = driver.find_element_by_css_selector('button[aria-label="下一頁"]').click()
    time.sleep(random.randint(7,20))
    print(i+1)  #印出已完成頁數
    
"""
目前的方法是一天一個excel檔抓取資料並儲存，
在網頁的地方可以直接改日期，不用重新在booking頁面選擇:
注意!!網址有輛個地方顯示日期，
"checkin_year=2022&checkin_month=5&checkin_monthday=07"的地方改才有用
但為確保成功，還是兩個地方都同時修改日期，
"""

C:\Users\User\AppData\Local\Temp/ipykernel_6564/2551738077.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\User\\chromedriver.exe")
C:\Users\User\AppData\Local\Temp/ipykernel_6564/2551738077.py:36: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  for hotel in driver.find_elements_by_css_selector('div[data-testid="title"]'):
C:\Users\User\AppData\Local\Temp/ipykernel_6564/2551738077.py:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  name = driver.find_element_by_id("hp_hotel_name_reviews").get_attribute("innerText")
C:\Users\User\AppData\Local\Temp/ipykernel_6564/2551738077.py:48: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  address = driver.find_element_by_css_sele

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


KeyboardInterrupt: 